In [1]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, regexp_replace, split, col, size, array_contains, isnan, when, count, array, reverse, udf, unix_timestamp, from_unixtime, date_format, format_number, length
from pyspark.sql.types import ArrayType, StringType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from scipy.stats import kurtosis, skew
import math
import folium
from geopy.geocoders import Nominatim
from itertools import chain

In [2]:
findspark.init()

In [3]:
spark_url = 'local'
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Project')\
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")\
        .getOrCreate()
spark.conf.set("spark.sql.csv.parser.multiLine", "true")

In [4]:
path = 'bangkok_traffy.csv'
df = spark.read.option("multiLine", "true").csv(path, header=True, inferSchema=True)
df.show(5)

+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|               type|        organization|             comment|               photo|         photo_after|            coords|             address|subdistrict|district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|                 {}|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13.79386|1867 จรัญสนิทวงศ์...|    บางพลัด| บางพล

In [5]:
null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|ticket_id|type|organization|comment|photo|photo_after|coords|address|subdistrict|district|province|timestamp|state|  star|count_reopen|last_activity|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|     2413|1550|        2640|   3911| 2089|      85624|  2019|   4433|       2089|    2092|    2393|     2233| 2026|164087|      120042|         2558|
+---------+----+------------+-------+-----+-----------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+



In [6]:
# Filter the DataFrame to include only rows with null values in the "column_name" column
def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row
    
ticket_id_null_df = df.filter(df.ticket_id.isNull())
coords_null_df = df.filter(df.coords.isNull())
address_null_df = df.filter(df.address.isNull())
print(check_first_null(ticket_id_null_df), "\n")
print(check_first_null(coords_null_df), "\n")
print(check_first_null(address_null_df), "\n")

Row(ticket_id=None, type='{ถนน}', organization='สำนักงาน ป.ป.ท.,เขตจอมทอง,ฝ่ายเทศกิจ เขตจอมทอง,ผอ.เขตจอมทอง (นายณัฐพงษ์),กลุ่มกรุงธนเหนือ (รองปลัดฯ เฉลิมพล)', comment=None, photo='https://storage.googleapis.com/traffy_public_bucket/TeamChadChart/corruption_photo2.png', photo_after='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-06/e9596093de70ae8abacd6574f26a2d0f4466fe9f.jpg', coords='100.45568,13.69103', address=None, subdistrict='บางขุนเทียน', district='จอมทอง', province='กรุงเทพมหานคร', timestamp='2022-06-09 23:34:34.98044+00', state='เสร็จสิ้น', star='5', count_reopen=None, last_activity='2022-06-10 11:02:34.607728+00') 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', organization=None, comment='"เคยดีใจมีสายสีน้ำเงินสถานี""แยกไฟฉาย""', photo=None, photo_after=None, coords=None, address=None, subdistrict=None, district=None, province=None, timestamp=None, state=None, star=None, count_reopen=None, last_activity=None) 

Row(ticket_id='2022-7DABXT', type='{สะพาน}', 

From above cells give us the first row with null value from each column selected (ticket_id, coords, address) to tell some relationship of those null value.
1. The ticket_id is null when the state='เสร็จสิ้น', it's mean we can drop this column significantly.
2. The address has null value 2 times more than coords. In the first null row we can see both of them are null. So it might tell that if no coords, no address too and not vice versa. We'll check in next step. 

In [7]:
sub = ['coords', 'address']
df = df.dropna(how='all', subset=sub)
sub_null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sub])
sub_null_counts.show()

+------+-------+
|coords|address|
+------+-------+
|     1|   2415|
+------+-------+



As we can see, the null coords most dissapear. So, we don't need to map any address to coords and we can also drop all remained null column.

In [8]:
df = df.dropna(how='all', subset=['coords'])
df = df.dropna(how='all', subset=['address'])

From this step, we'll use df_use to be a data for ML

In [9]:
# change state to boolean 
df_use = df.withColumn('state', when(df.state == 'เสร็จสิ้น', 1).otherwise(0))

# change type to list
df_use = df_use.withColumn("type", split(regexp_replace("type", "[{}]", ""), ","))
df_use = df_use.dropna(how='all', subset=['type'])

# change coords to pair and swap them into format [latitude, longtitude]
flatten = udf(lambda x: list(chain.from_iterable(x)), ArrayType(StringType()))
df_use = df_use.withColumn('coords', array(reverse(split(df.coords, ','))))
df_use = df_use.withColumn('coords', flatten('coords'))

# change timestamp and last_activity
df_use = df_use.withColumn('timestamp', from_unixtime(unix_timestamp(col('timestamp'), 'yyyy-MM-dd HH:mm')))
df_use = df_use.withColumn('last_activity', from_unixtime(unix_timestamp(col('last_activity'), 'yyyy-MM-dd HH:mm')))
df_use = df_use.withColumn('time_diff', (unix_timestamp(col('last_activity')) - unix_timestamp(col('timestamp'))) / 86400)
df_use = df_use.withColumn('time_diff', format_number(col('time_diff'), 0))
df_use.show(5)

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+--------------------+-------------------+-----+----+------------+-------------------+---------+
|  ticket_id|                type|        organization|             comment|               photo|         photo_after|              coords|             address|subdistrict|district|            province|          timestamp|state|star|count_reopen|      last_activity|time_diff|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+--------------------+-------------------+-----+----+------------+-------------------+---------+
|2021-9LHDM6|                  []|                null|            ไม่มีภาพ|https://storage.g...|                null|[13.79386, 100.48...|1867 จรัญสนิทวงศ์...|    บางพล

In [10]:
df_use.groupBy(col('star')).count().show()

+--------------------+------+
|                star| count|
+--------------------+------+
|2022-07-20 10:44:...|     1|
|                   3| 10060|
|2022-08-03 14:58:...|     1|
|                null|160084|
|                   5| 45910|
|2023-01-24 01:23:...|     1|
|2022-08-03 06:06:...|     1|
|2022-08-30 20:17:...|     1|
|2023-02-20 10:27:...|     1|
|                   1| 13519|
|           เสร็จสิ้น|    12|
|2022-07-05 11:38:...|     1|
|                   4| 20335|
|2022-07-05 11:40:...|     1|
|2022-07-17 16:58:...|     1|
|2022-06-29 07:25:...|     1|
|2022-07-25 13:21:...|     1|
|                   2|  4544|
+--------------------+------+



In [11]:
def check_first_null(filtered_df):
    # Check if the filtered DataFrame is empty
    if filtered_df.count() == 0:
        return f"No null values in {filtered_df}."
    else:
        first_row = filtered_df.head()
        return first_row
    
star_null_df = df_use.filter(df_use.star.isNull())
print(check_first_null(star_null_df), "\n")

Row(ticket_id='2021-9LHDM6', type=[''], organization=None, comment='ไม่มีภาพ', photo='https://storage.googleapis.com/traffy_public_bucket/attachment/2021-09/ba04e344d8011ae51fdcf6db72de76780575d10c.jpg', photo_after=None, coords=['13.79386', '100.48661'], address='1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุงเทพมหานคร 10700 ประเทศไทย', subdistrict='บางพลัด', district='บางพลัด', province='กรุงเทพมหานคร', timestamp='2021-09-01 10:44:00', state=0, star=None, count_reopen=None, last_activity='2022-02-22 04:59:00', time_diff='174') 



In [12]:
df_use.filter(length('star') > 1).groupBy(col('star')).count().show()

+--------------------+-----+
|                star|count|
+--------------------+-----+
|2022-07-20 10:44:...|    1|
|2022-08-03 14:58:...|    1|
|2023-01-24 01:23:...|    1|
|2022-08-03 06:06:...|    1|
|2022-08-30 20:17:...|    1|
|2023-02-20 10:27:...|    1|
|           เสร็จสิ้น|   12|
|2022-07-05 11:38:...|    1|
|2022-07-05 11:40:...|    1|
|2022-07-17 16:58:...|    1|
|2022-06-29 07:25:...|    1|
|2022-07-25 13:21:...|    1|
+--------------------+-----+



In [13]:
first_element = udf(lambda x: x[0], StringType())
# df_exploded = df_use.withColumn('type', first_element(df_use['type']))
df_exploded = df_use.withColumn('type', explode(df_use['type']))
df_exploded = df_exploded.filter("type != ''")

df_exploded.show()

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+-----+----+------------+-------------------+---------+
|  ticket_id|       type|        organization|             comment|               photo|         photo_after|              coords|             address|   subdistrict| district|            province|          timestamp|state|star|count_reopen|      last_activity|time_diff|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+---------+--------------------+-------------------+-----+----+------------+-------------------+---------+
|2021-FYJTFP|  ความสะอาด|          เขตบางซื่อ|             ขยะเยอะ|https://storage.g...|                null|[13.81865, 100.53...|12/14 ถนน กรุงเทพ...|          null|     null|       ก

In [14]:
df_t = df_exploded.withColumn("latitude", col("coords")[0]) \
       .withColumn("longitude", col("coords")[1]) \
       .drop("coords")
df_t = df_t.withColumn("latitude", col("latitude").cast('double'))
df_t = df_t.withColumn("longitude", col("longitude").cast('double'))

In [15]:
null_latitude = df_t.filter(df_t.latitude.isNull())
null_longitude = df_t.filter(df_t.longitude.isNull())
print(check_first_null(null_latitude), "\n")
print(check_first_null(null_longitude), "\n")

Row(ticket_id='จักขอบคุณอย่างยิ่ง"', type='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-02/5d20ecb864e916af948b4090ce9405be6f0b2f04.jpg', organization=None, comment='100.46930,13.75503', photo='1186 ถ. พรานนก แขวง บ้านช่างหล่อ เขตบางกอกน้อย กรุงเทพมหานคร 10700 ประเทศไทย', photo_after='บางขุนศรี', address='กรุงเทพมหานคร', subdistrict='2022-02-06 16:59:19.251528+00', district='กำลังดำเนินการ', province=None, timestamp=None, state=0, star=None, count_reopen=None, last_activity=None, time_diff=None, latitude=None, longitude=None) 

Row(ticket_id='จักขอบคุณอย่างยิ่ง"', type='https://storage.googleapis.com/traffy_public_bucket/attachment/2022-02/5d20ecb864e916af948b4090ce9405be6f0b2f04.jpg', organization=None, comment='100.46930,13.75503', photo='1186 ถ. พรานนก แขวง บ้านช่างหล่อ เขตบางกอกน้อย กรุงเทพมหานคร 10700 ประเทศไทย', photo_after='บางขุนศรี', address='กรุงเทพมหานคร', subdistrict='2022-02-06 16:59:19.251528+00', district='กำลังดำเนินการ', province=None, timestamp=

In [16]:
drop_list = ['type', 'latitude', 'longitude', 'subdistrict', 'district', 'timestamp', 'last_activity', 'organization', 'star', 'time_diff']
for e in drop_list:
    df_t = df_t.dropna(how='all', subset=[e])

## Classification

In [17]:
def train_random_forest(df, label_col, numeric_col,  quantitative_col):
    stages = []

    # For label
    label_indexer = StringIndexer(inputCol=label_col, outputCol='label')
    indexer_model = label_indexer.fit(df)
    df = indexer_model.transform(df)

    # For features
    for column_name in quantitative_col:
        stringIndexer = StringIndexer(inputCol=column_name, outputCol=column_name+ "_index")
        stages.append(stringIndexer)
        oneHotEncoder = OneHotEncoder(inputCol=column_name+ "_index", outputCol=column_name + "_onehot")
        stages.append(oneHotEncoder)

    quantitative_cols = [s + "_onehot" for s in quantitative_col]
    assembler = VectorAssembler(inputCols=numeric_col + quantitative_cols, outputCol='features')
    stages.append(assembler)

    pipeline = Pipeline(stages=stages)
    preprocessed_data = pipeline.fit(df).transform(df)

    # Split data into training and testing sets
    train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3], seed=123)

    # Train Decision Tree classifier
    dt = RandomForestClassifier(labelCol='label', featuresCol='features', maxDepth=6)
    dt_model = dt.fit(train_data)
    
    # Make predictions on test set
    predictions = dt_model.transform(test_data)
    label_reverse = IndexToString(inputCol="prediction", outputCol="predicted_result", labels=indexer_model.labels)
    predictions_res = label_reverse.transform(predictions)

    # Evaluate performance
    evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
    accuracy = evaluator.evaluate(predictions)
    print('Accuracy:', accuracy)

    return predictions_res

In [23]:
numeric_col = []
quantitative_col = ['type', 'organization', 'star', 'time_diff']
df_select = df_t.select(numeric_col + quantitative_col)
try:
    predictions_res = train_random_forest(df_select, 'star', numeric_col,  quantitative_col)
except Exception as e:
    print(e)

Accuracy: 0.4845711115733999


In [19]:
predictions_res.groupBy(col('predicted_result')).count().show()

+----------------+-----+
|predicted_result|count|
+----------------+-----+
|               5|28756|
|               4|   86|
+----------------+-----+



## Regression

In [20]:
def train_linear_regression(df, label_col, numeric_col,  quantitative_col):
    stages = []
    df_select = df.select(numeric_col + quantitative_col)

    # For label
    label_indexer = StringIndexer(inputCol=label_col, outputCol='label')
    indexer_model = label_indexer.fit(df_select)
    df_select = indexer_model.transform(df_select)

    # For features
    for column_name in quantitative_col:
        stringIndexer = StringIndexer(inputCol=column_name, outputCol=column_name+ "_index")
        stages.append(stringIndexer)
        oneHotEncoder = OneHotEncoder(inputCol=column_name+ "_index", outputCol=column_name + "_onehot")
        stages.append(oneHotEncoder)

    quantitative_cols = [s + "_onehot" for s in quantitative_col]
    assembler = VectorAssembler(inputCols=numeric_col + quantitative_cols, outputCol='features')
    stages.append(assembler)

    pipeline = Pipeline(stages=stages)
    preprocessed_data = pipeline.fit(df_select).transform(df_select)

    # Split data into training and testing sets
    train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3], seed=123)

    # Train Decision Tree classifier
    poisson = GeneralizedLinearRegression(family='poisson', link='log', maxIter=10, regParam=0.3)
    dt_model = poisson.fit(train_data, label_col)
    predictions = dt_model.transform(test_data)

    # Make predictions on test set
    label_reverse = IndexToString(inputCol="prediction", outputCol="predicted_result", labels=indexer_model.labels)
    predictions_res = label_reverse.transform(predictions)

    # Evaluate the performance of the model using mean squared error
    from pyspark.ml.evaluation import RegressionEvaluator
    evaluator = RegressionEvaluator(predictionCol='prediction', labelCol=label_col, metricName='mse')
    mse = evaluator.evaluate(predictions)
    print('Mean Squared Error:', mse)

    return predictions_res

In [21]:
df_t.select('timestamp').show(truncate=False)

+-------------------+
|timestamp          |
+-------------------+
|2021-09-19 14:56:00|
|2021-09-19 14:56:00|
|2021-12-09 12:29:00|
|2021-12-09 12:29:00|
|2021-12-18 14:50:00|
|2021-12-22 10:15:00|
|2021-12-28 03:59:00|
|2022-01-28 04:09:00|
|2022-02-01 18:11:00|
|2022-02-04 12:06:00|
|2022-02-06 06:23:00|
|2022-02-15 02:47:00|
|2022-02-16 15:45:00|
|2022-02-16 15:45:00|
|2022-02-26 09:20:00|
|2022-02-26 14:36:00|
|2022-05-18 05:25:00|
|2022-05-22 17:47:00|
|2022-05-24 11:21:00|
|2022-05-25 04:02:00|
+-------------------+
only showing top 20 rows



In [1]:
spark.stop()

NameError: name 'spark' is not defined